In [1]:
import os
import cv2
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import shutil


In [34]:

def create_video_from_images(image_folder, output_video, fps):
    images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
    images.sort(key=lambda x: int(x.split('_')[1].split('.')[0]))  # Sort images numerically

    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))

    cv2.destroyAllWindows()
    video.release()

# Specify the folder containing the images, the output video path, and frames per second (fps)
image_folder = "./combined"
output_video = "./video/combined.mp4"
fps = 15  # Frames per second

create_video_from_images(image_folder, output_video, fps)

In [4]:
def combine_images(image_paths, direction='horizontal'):
  images = [Image.open(p) for p in image_paths]
  
  if direction == 'horizontal':
      total_width = sum(img.width for img in images)
      max_height = max(img.height for img in images)
      combined = Image.new('RGB', (total_width, max_height))
      x_offset = 0
      for img in images:
          combined.paste(img, (x_offset, 0))
          x_offset += img.width
  else:  # vertical
      max_width = max(img.width for img in images)
      total_height = sum(img.height for img in images)
      combined = Image.new('RGB', (max_width, total_height))
      y_offset = 0
      for img in images:
          combined.paste(img, (0, y_offset))
          y_offset += img.height

  return combined

In [15]:
# def combine_images_square_with_title(image_paths, title="My Gallery", header_height=80):
#     if len(image_paths) != 4:
#         raise ValueError("Exactly 4 images are required to make a square grid.")

#     images = [Image.open(p) for p in image_paths]

#     # Resize all images to the same size (use first image size)
#     width, height = images[0].size
#     images = [img.resize((width, height)) for img in images]

#     # Create a new blank image (2x2 grid + header slice)
#     combined = Image.new('RGB', (width * 2, height * 2 + header_height), color="white")

#     # --- Draw header background ---
#     draw = ImageDraw.Draw(combined)
#     draw.rectangle([(0, 0), (width * 2, header_height)], fill="lightgray")

#     # --- Add title text ---
#     try:
#         font = ImageFont.truetype("arial.ttf", 36)  # Use system font
#     except:
#         font = ImageFont.load_default()

#     text_w, text_h = draw.textsize(title, font=font)
#     text_x = (width * 2 - text_w) // 2
#     text_y = (header_height - text_h) // 2
#     draw.text((text_x, text_y), title, fill="black", font=font)

#     # --- Paste images in 2x2 grid ---
#     combined.paste(images[0], (0, header_height))             # top-left
#     combined.paste(images[1], (width, header_height))         # top-right
#     combined.paste(images[2], (0, height + header_height))    # bottom-left
#     combined.paste(images[3], (width, height + header_height))# bottom-right

#     return combined

def combine_images_square_with_title(image_paths, title="My Collage", font_size=40, font_path=None):
    if len(image_paths) != 4:
        raise ValueError("Exactly 4 images are required to make a square grid.")

    images = [Image.open(p) for p in image_paths]

    # Resize all images to the same size (use the size of the first image)
    width, height = images[0].size
    images = [img.resize((width, height)) for img in images]

    # Reserve extra space on top for the title
    title_height = font_size + 60  # space for text + padding
    combined = Image.new('RGB', (width * 2, height * 2 + title_height), color="white")

    # Draw title
    draw = ImageDraw.Draw(combined)
    try:
        font = ImageFont.truetype(font_path if font_path else "arial.ttf", font_size)
    except:
        font = ImageFont.load_default()

    # Use textbbox to get size
    bbox = draw.textbbox((0, 0), title, font=font)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]

    text_x = (combined.width - text_width) // 2
    text_y = (title_height - text_height) // 2
    draw.text((text_x, text_y), title, fill="black", font=font)

    # Paste images in 2x2 grid below the title
    combined.paste(images[0], (0, title_height))                   # top-left
    combined.paste(images[1], (width, title_height))               # top-right
    combined.paste(images[2], (0, height + title_height))          # bottom-left
    combined.paste(images[3], (width, height + title_height))      # bottom-right

    return combined


def combine_images_square(image_paths):
    if len(image_paths) != 4:
        raise ValueError("Exactly 4 images are required to make a square grid.")

    images = [Image.open(p) for p in image_paths]

    # Resize all images to the same size (optional, here we use the size of the first image)
    width, height = images[0].size
    images = [img.resize((width, height)) for img in images]

    # Create a new blank image (2x2 grid)
    combined = Image.new('RGB', (width * 2, height * 2))

    # Paste images in 2x2 grid
    combined.paste(images[0], (0, 0))             # top-left
    combined.paste(images[1], (width, 0))         # top-right
    combined.paste(images[2], (0, height))        # bottom-left
    combined.paste(images[3], (width, height))    # bottom-right

    return combined

In [6]:
def format_frame_number(frame_number):
    return f"{frame_number:03d}"

format_frame_number(1)

'001'

In [21]:
combine_images_square_with_title([
  "frames-gd/frame_000.png",
  "frames-sgd/frame_000.png",
  "frames-mini-batch-5-sgd/frame_000.png",
  "frames-mini-batch-10-sgd/frame_000.png"
], title="Visual Comparison of Gradient Descent Methods\n                Epoch {(sgd_i // gd_iters) + 1}", font_size=60).show()


In [9]:
SAMPLES = 50
MAX_EPOCHS = 10

frames_gd = np.arange(0, 10)
frames_gd_extended = np.repeat(frames_gd, SAMPLES) 

frames_mini_batch_5_sgd = np.arange(0, 100)
frames_mini_batch_5_sgd_extended = np.repeat(frames_mini_batch_5_sgd, 5)

frames_mini_batch_10_sgd = np.arange(0, 50)
frames_mini_batch_10_sgd_extended = np.repeat(frames_mini_batch_10_sgd, 10)

frames_sgd_extended = np.arange(0, 500)

In [31]:
frames_dir = './combined'
if os.path.exists(frames_dir):
    shutil.rmtree(frames_dir)
os.makedirs(frames_dir)

In [33]:
for sgd_i in np.arange(0, 500):
  sgd_file_name = f"frames-sgd/frame_{format_frame_number(sgd_i)}.png"
  
  gd_iters = SAMPLES // 1
  gd_file_name = f"frames-gd/frame_{format_frame_number(sgd_i // gd_iters)}.png"

  mini_batch_5_iters = SAMPLES // 10
  mini_batch_5_file_name = f"frames-mini-batch-5-sgd/frame_{format_frame_number(sgd_i // mini_batch_5_iters)}.png"

  mini_batch_10_iters = SAMPLES // 5
  mini_batch_10_file_name = f"frames-mini-batch-10-sgd/frame_{format_frame_number(sgd_i // mini_batch_10_iters)}.png"

  title = f'Visual Comparison of Gradient Descent Methods\n                                Epoch {(sgd_i // gd_iters) + 1}'
  
  combined = combine_images_square_with_title([gd_file_name, mini_batch_10_file_name, mini_batch_5_file_name, sgd_file_name], title, font_size=50)
  combined.save(f"combined/frame_{format_frame_number(sgd_i)}.png")
 